In [19]:
import numpy as np
from helpers import *

In [21]:
def initialize_parameters(layers_size):
    """
    Returns parameters ('W1', 'b1', W2', 'b2', ...)
    """
    
    parameters = {}
    for l in range(1, len(layers_size)):
        parameters['W' + str(l)] = np.random.randn(layers_size[l], layers_size[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layers_size[l], 1))
    
    return parameters

In [22]:
def linearize_and_activate_forward(A_previous, W, b, activation_function='relu'):
    """
    Returns A
    """
    
    Z = np.dot(W, A_previous) + b
    
    if activation == 'relu':
        A = relu(Z)
    elif activation == 'sigmoid':
        A = sigmoid(Z)
    
    return A

In [ ]:
def calculate_cost(Y_hat, Y):
    """
    """
    
    
    log_probs = np.dot(Y, np.log(Y_hat)) + np.dot(1 - Y, np.log(1 - Y_hat))
    cost = -